In [1]:
import numpy as np
import pandas as pd
import scipy.io
from scipy import stats

# import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
import os
import pickle
import time
from IPython.display import clear_output

import logging
from datetime import datetime

# cell filter
visually responsive & image driven 

In [2]:
dir_inter = r'Z:\All_Staff\home\lan\Data\2P_images\mat_inter/'.replace('\\', '/')
dir_file = dir_inter + 'adp_dataset_master.xlsx'
data_info = pd.read_excel(dir_file)
data_info.head()

meta = data_info[(data_info.paradigm == 'grating_lindsey_miaomiao') 
                #  & (data_info.area == 'LM')
                #  & (data_info.gcamp == '6s') # avoid mixing in gcamp8f
                #  & ((data_info.cellpose_seg == True) | (data_info.manual_seg == True)) # ensure segmentation
                 ]

meta = meta.reset_index(drop=True)
nset = meta.shape[0]
print(meta.area.value_counts())
meta

V1    12
Name: area, dtype: int64


,mouse,date,area,depth,num,cellpose_seg,manual_seg,paradigm,gcamp,AWS,note
0,674,170324,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6f,NaN,TC and input already concat-ed 002-003. see in...
1,689,170323,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6f,NaN,NaN
2,696,170323,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6s,NaN,NaN
3,684,170327,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6f,NaN,NaN
4,711,170503,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6s,NaN,NaN
5,712,170503,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6s,NaN,NaN
6,574,170510,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6s,NaN,NaN
7,720,170808,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6s,NaN,NaN
8,738,170810,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6f,NaN,NaN
9,739,170811,V1,NaN,002-003,NaN,True,grating_lindsey_miaomiao,6f,NaN,NaN


In [6]:
session_mode = 'single session' # then irun must be appended to dir_sub !
# session_mode = 'multi session' # then irun must be appended to dir_sub !

## set up logging
plt.set_loglevel(level='warning') # turn off matplotlib debug logging
logging.basicConfig(filename='C:/Users/ll357/Documents/inter/data/vis_driven.log', level=logging.DEBUG)
logging.info('\n\n\n\n\n') # log several linebreaks to separate different runs
logging.info(str(datetime.now()))

for i in np.arange(nset): # TODO: this overwrites vis-driven.pickle multiple times for multisess data, fix later
    # clear_output(wait=True)
    logging.info(f'set {i+1} of {nset}')
    mouse = meta.iloc[i].mouse.astype(str)
    date = meta.iloc[i].date.astype(str)
    area = meta.iloc[i].area
    irun = meta.iloc[i].num # already a string due to concat lindsey's grating 8ori data
    logging.info(f'{mouse} {date} {irun} {area}')

    ## check if resp_base_trialwise exists
    ## load data
    logging.info('waiting for mat')
    print('waiting for mat')
    while True:
        try:
            dir_sub = area + '_i' + mouse + '_' + date
            if session_mode == 'single session':
                dir_sub += '_' + irun # else: multi sess, dont append irun
            print(dir_sub)
            try: # manual seg data has no suffix '_cellpose'
                dfof_trialwise = scipy.io.loadmat(os.path.join(dir_inter, dir_sub, 'resp_base_trialwise' + '.mat'))
            except: # cellpose seg data has suffix '_cellpose'
                dir_sub += '_cellpose'
                dfof_trialwise = scipy.io.loadmat(os.path.join(dir_inter, dir_sub, 'resp_base_trialwise' + '.mat'))
            logging.info('mat loaded')
            break
        except:
            time.sleep(60)
            clear_output(wait=True)
            continue

    if len(dfof_trialwise['dfof_ad_trial'].shape) == 2: # if contains only one ISI, likely 250
        isi_mode = 'only one isi'
        print(f'isi_mode is {isi_mode} (only one isi, grat_SF or bunny)')
        print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0].shape)
        dfof_ad_trial = dfof_trialwise['dfof_ad_trial'] # do not subtract baseline here: stim resp will be compared to base resp
        dfof_base_trial = dfof_trialwise['dfof_base_trial']

    if len(dfof_trialwise['dfof_ad_trial'].shape) == 3: # if contains multiple ISI, likely inf-750-250
        isi_mode = 'grat_8ori_3isi'
        print(f'isi_mode is {isi_mode} (grating with 8 orientations and 3 ISI)')
        print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0,0].shape)
        dfof_tg_trial = dfof_trialwise['dfof_tg_trial'][:,:,0] # only use the first ISI, aka no adapter trials
        dfof_base2_trial = dfof_trialwise['dfof_base2_trial'][:,:,0]
        print('to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. \n 8 possible target orientations will cover all needed cells')

    ## stats
    if isi_mode == 'only one isi':
        print(isi_mode)

        ncell = dfof_ad_trial.shape[0]
        nstim = dfof_ad_trial.shape[1]

        ## according to Ohki 2020
        # p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
        # p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
        p_ttest = np.ones((ncell, nstim)) * np.nan
        evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

        for icell in tqdm(np.arange(ncell)):
            base_cell_anova = np.concatenate([dfof_base_trial[icell, stim] for stim in range(nstim)])
            stim_cell_anova = [] 
            for istim in np.arange(nstim):
                stim_cell_anova.append(np.array(dfof_ad_trial[icell, istim]).flatten())

                base_cell = dfof_base_trial[icell, istim]
                stim_cell = dfof_ad_trial[icell, istim]
                _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
                p_ttest[icell, istim] = p_ttest_i

                evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
                evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
                evoked[icell, istim] = evoked_i
            
            # _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
            # p_anova[icell] = p_anova_cell
            # _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
            # p_kruskal[icell] = p_kruskal_cell

    elif isi_mode == 'grat_8ori_3isi':
        print(isi_mode)

        ncell = dfof_tg_trial.shape[0]
        nstim = dfof_tg_trial.shape[1]

        # according to Ohki 2020
        # p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
        # p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
        p_ttest = np.ones((ncell, nstim)) * np.nan
        evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

        for icell in tqdm(np.arange(ncell)):
            base_cell_anova = np.concatenate([dfof_base2_trial[icell, stim] for stim in range(nstim)])
            stim_cell_anova = [] 
            for istim in np.arange(nstim):
                stim_cell_anova.append(np.array(dfof_tg_trial[icell, istim]).flatten())

                base_cell = dfof_base2_trial[icell, istim]
                stim_cell = dfof_tg_trial[icell, istim]
                _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
                p_ttest[icell, istim] = p_ttest_i

                evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
                evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
                evoked[icell, istim] = evoked_i
            
            # _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
            # p_anova[icell] = p_anova_cell
            # _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
            # p_kruskal[icell] = p_kruskal_cell


    # cells responsive to image i: pass visually driven (anova OR kruskal) AND t-test AND amp threshold for *this* image
    p_sig = 0.05
    evoked_thresh = 0.05
    img_driven = (p_ttest < p_sig) & (evoked > evoked_thresh)
    img_driven_msg = f'{img_driven.sum()} cells are image driven - with overlap between images, \n\
        proportion {np.round(img_driven.sum() / (ncell*nstim), 2)} out of {ncell*nstim} cell-stim combos. \n\
        1-N image evokes resp from {np.sum(img_driven, axis=0)} cells'
    logging.info(img_driven_msg)
    print(img_driven_msg)

    t = np.sum(img_driven, axis=1)
    logging.info(f'img driven cells are driven by {t[t>0]} images')

    # visually driven cells: NOTE changed to: driven by any image (pass t-test to any stim2, but with bonferroni correction) AND amp threshold. 
    loosen_sig = 6
    p_bonferroni_corrected = loosen_sig * p_sig / nstim # number of possible stim2
    vis_driven = ((np.sum(p_ttest < p_bonferroni_corrected, axis=1) > 0) # any stim2 passes t-test
                & (np.sum(evoked > evoked_thresh, axis=1) > 0) # any stim2 evokes resp > thresh
                )
    vis_driven_msg = f'{vis_driven.sum()} cells are visually driven, \n\
        proportion {np.round(vis_driven.sum()/ncell, 2)} out of {ncell} cells'
    logging.info(vis_driven_msg)
    print(vis_driven_msg)

    # break

    ## save
    vis_driven_re = {}
    # vis_driven_re['p_anova'] = p_anova
    # vis_driven_re['p_kruskal'] = p_kruskal
    vis_driven_re['p_ttest'] = p_ttest
    vis_driven_re['evoked'] = evoked
    vis_driven_re['p_sig'] = p_sig
    vis_driven_re['p_bonferroni_corrected'] = p_bonferroni_corrected
    vis_driven_re['vis_driven'] = vis_driven
    vis_driven_re['img_driven'] = img_driven

    os.chdir(os.path.join(dir_inter, dir_sub))
    print(os.getcwd())
    with open('vis_driven_ttest_bonferroni.pickle', 'wb') as f:
        pickle.dump(vis_driven_re, f)
    
    # break

waiting for mat
V1_i674_170324_002-003
isi_mode is grat_8ori_3isi (grating with 8 orientations and 3 ISI)
(73, 8, 3) (16, 1)
to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. 
 8 possible target orientations will cover all needed cells
grat_8ori_3isi


100%|██████████| 73/73 [00:00<00:00, 611.66it/s]


61 cells are image driven - with overlap between images, 
        proportion 0.1 out of 584 cell-stim combos. 
        1-N image evokes resp from [10  3  7  7 11  4  9 10] cells
41 cells are visually driven, 
        proportion 0.56 out of 73 cells
Z:\All_Staff\home\lan\Data\2P_images\mat_inter\V1_i674_170324_002-003
waiting for mat
V1_i689_170323_002-003
isi_mode is grat_8ori_3isi (grating with 8 orientations and 3 ISI)
(19, 8, 3) (14, 1)
to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. 
 8 possible target orientations will cover all needed cells
grat_8ori_3isi


100%|██████████| 19/19 [00:00<00:00, 558.82it/s]


18 cells are image driven - with overlap between images, 
        proportion 0.12 out of 152 cell-stim combos. 
        1-N image evokes resp from [3 2 3 4 1 1 0 4] cells
9 cells are visually driven, 
        proportion 0.47 out of 19 cells
Z:\All_Staff\home\lan\Data\2P_images\mat_inter\V1_i689_170323_002-003
waiting for mat
V1_i696_170323_002-003
isi_mode is grat_8ori_3isi (grating with 8 orientations and 3 ISI)
(73, 8, 3) (17, 1)
to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. 
 8 possible target orientations will cover all needed cells
grat_8ori_3isi


...

48 cells are image driven - with overlap between images, 
        proportion 0.17 out of 280 cell-stim combos. 
        1-N image evokes resp from [ 3  2  6  7  7  8  5 10] cells
32 cells are visually driven, 
        proportion 0.91 out of 35 cells
Z:\All_Staff\home\lan\Data\2P_images\mat_inter\V1_i739_170811_002-003
waiting for mat
V1_i745_170816_002-003
isi_mode is grat_8ori_3isi (grating with 8 orientations and 3 ISI)
(66, 8, 3) (17, 1)
to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. 
 8 possible target orientations will cover all needed cells
grat_8ori_3isi


100%|██████████| 66/66 [00:00<00:00, 580.56it/s]


90 cells are image driven - with overlap between images, 
        proportion 0.17 out of 528 cell-stim combos. 
        1-N image evokes resp from [13 12 12  0 15 12 16 10] cells
60 cells are visually driven, 
        proportion 0.91 out of 66 cells
Z:\All_Staff\home\lan\Data\2P_images\mat_inter\V1_i745_170816_002-003
waiting for mat
V1_i746_170826_002-003
isi_mode is grat_8ori_3isi (grating with 8 orientations and 3 ISI)
(49, 8, 3) (16, 1)
to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. 
 8 possible target orientations will cover all needed cells
grat_8ori_3isi


100%|██████████| 49/49 [00:00<00:00, 571.16it/s]

69 cells are image driven - with overlap between images, 
        proportion 0.18 out of 392 cell-stim combos. 
        1-N image evokes resp from [ 6  9 13  0 13  9  6 13] cells
45 cells are visually driven, 
        proportion 0.92 out of 49 cells
Z:\All_Staff\home\lan\Data\2P_images\mat_inter\V1_i746_170826_002-003


In [39]:
pickle_dir = r'Z:\All_Staff\home\lan\Data\2P_images\mat_inter\V1_i1337_210120_003_cellpose/'.replace('\\', '/')
with open(pickle_dir+'vis_driven_ttest_bonferroni.pickle', 'rb') as f:
    vis_driven_re = pickle.load(f)
vis_driven_re['vis_driven'].shape

import scipy.io as sio
os.chdir(r'C:\Users\ll357\Documents\inter\results\tuning curve bias san check'.replace('\\', '/'))
sio.savemat('vis_driven_ttest_bonferroni.mat', vis_driven_re)

In [35]:
f'{vis_driven.sum()} cells are visually driven, \n\
        proportion {np.round(vis_driven.sum()/ncell, 2)} out of {ncell} cells'

'64 cells are visually driven, \n        proportion 0.62 out of 103 cells'

In [17]:
# session_mode = 'single session' # then irun must be appended to dir_sub !
# # session_mode = 'multi session' # then irun must be appended to dir_sub !

# ## set up logging
# plt.set_loglevel(level='warning') # turn off matplotlib debug logging
# logging.basicConfig(filename='C:/Users/ll357/Documents/inter/data/vis_driven.log', level=logging.DEBUG)
# logging.info('\n\n\n\n\n') # log several linebreaks to separate different runs
# logging.info(str(datetime.now()))

# for i in np.arange(nset): # TODO: this overwrites vis-driven.pickle multiple times for multisess data, fix later
#     # clear_output(wait=True)
#     logging.info(f'set {i+1} of {nset}')
#     mouse = meta.iloc[i].mouse.astype(str)
#     date = meta.iloc[i].date.astype(str)
#     area = meta.iloc[i].area
#     irun = '00' + meta.iloc[i].num.astype(int).astype(str)
#     logging.info(f'{mouse} {date} {irun} {area}')

#     ## check if resp_base_trialwise exists
#     ## load data
#     logging.info('waiting for mat')
#     print('waiting for mat')
#     while True:
#         try:
#             dir_sub = area + '_i' + mouse + '_' + date
#             if session_mode == 'single session':
#                 dir_sub += '_' + irun # else: multi sess, dont append irun
#             print(dir_sub)
#             try: # manual seg data has no suffix '_cellpose'
#                 dfof_trialwise = scipy.io.loadmat(os.path.join(dir_inter, dir_sub, 'resp_base_trialwise' + '.mat'))
#             except: # cellpose seg data has suffix '_cellpose'
#                 dir_sub += '_cellpose'
#                 dfof_trialwise = scipy.io.loadmat(os.path.join(dir_inter, dir_sub, 'resp_base_trialwise' + '.mat'))
#             logging.info('mat loaded')
#             break
#         except:
#             time.sleep(60)
#             clear_output(wait=True)
#             continue

#     if len(dfof_trialwise['dfof_ad_trial'].shape) == 2: # if contains only one ISI, likely 250
#         isi_mode = 'only one isi'
#         print(f'isi_mode is {isi_mode} (only one isi, grat_SF or bunny)')
#         print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0].shape)
#         dfof_ad_trial = dfof_trialwise['dfof_ad_trial'] # do not subtract baseline here: stim resp will be compared to base resp
#         dfof_base_trial = dfof_trialwise['dfof_base_trial']

#     if len(dfof_trialwise['dfof_ad_trial'].shape) == 3: # if contains multiple ISI, likely inf-750-250
#         isi_mode = 'grat_8ori_3isi'
#         print(f'isi_mode is {isi_mode} (grating with 8 orientations and 3 ISI)')
#         print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0,0].shape)
#         dfof_tg_trial = dfof_trialwise['dfof_tg_trial'][:,:,0] # only use the first ISI, aka no adapter trials
#         dfof_base2_trial = dfof_trialwise['dfof_base2_trial'][:,:,0]
#         print('to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. \n 8 possible target orientations will cover all needed cells')

#     ## stats
#     if isi_mode == 'only one isi':
#         print(isi_mode)

#         ncell = dfof_ad_trial.shape[0]
#         nstim = dfof_ad_trial.shape[1]

#         ## according to Ohki 2020
#         p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
#         p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
#         p_ttest = np.ones((ncell, nstim)) * np.nan
#         evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

#         for icell in tqdm(np.arange(ncell)):
#             base_cell_anova = np.concatenate([dfof_base_trial[icell, stim] for stim in range(nstim)])
#             stim_cell_anova = [] 
#             for istim in np.arange(nstim):
#                 stim_cell_anova.append(np.array(dfof_ad_trial[icell, istim]).flatten())

#                 base_cell = dfof_base_trial[icell, istim]
#                 stim_cell = dfof_ad_trial[icell, istim]
#                 _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
#                 p_ttest[icell, istim] = p_ttest_i

#                 evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
#                 evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
#                 evoked[icell, istim] = evoked_i
            
#             _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#             p_anova[icell] = p_anova_cell
#             _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#             p_kruskal[icell] = p_kruskal_cell

#     elif isi_mode == 'grat_8ori_3isi':
#         print(isi_mode)

#         ncell = dfof_tg_trial.shape[0]
#         nstim = dfof_tg_trial.shape[1]

#         ## according to Ohki 2020
#         p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
#         p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
#         p_ttest = np.ones((ncell, nstim)) * np.nan
#         evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

#         for icell in tqdm(np.arange(ncell)):
#             base_cell_anova = np.concatenate([dfof_base2_trial[icell, stim] for stim in range(nstim)])
#             stim_cell_anova = [] 
#             for istim in np.arange(nstim):
#                 stim_cell_anova.append(np.array(dfof_tg_trial[icell, istim]).flatten())

#                 base_cell = dfof_base2_trial[icell, istim]
#                 stim_cell = dfof_tg_trial[icell, istim]
#                 _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
#                 p_ttest[icell, istim] = p_ttest_i

#                 evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
#                 evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
#                 evoked[icell, istim] = evoked_i
            
#             _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#             p_anova[icell] = p_anova_cell
#             _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#             p_kruskal[icell] = p_kruskal_cell

#     # visually driven cells: pass (anova OR kruskal) AND amp threshold for >=1 image
#     p_sig = 0.05
#     vis_driven = ((p_anova < p_sig) | (p_kruskal < p_sig)) & (sum(evoked.T > 0.1) > 0).reshape(-1, 1)
#     logging.info(f'{vis_driven.sum()} cells are visually driven, \n\
#         proportion {np.round(vis_driven.sum()/ncell, 2)} out of {ncell} cells')

#     # cells responsive to image i: pass visually driven (anova OR kruskal) AND t-test AND amp threshold for *this* image
#     img_driven = vis_driven & (p_ttest < p_sig) & (evoked > 0.1)
#     logging.info(f'{img_driven.sum()} cells are image driven - with overlap between images, \n\
#         proportion {np.round(img_driven.sum() / (ncell*nstim), 2)} out of {ncell*nstim} cell-stim combos. \n\
#         1-N image evokes resp from {np.sum(img_driven, axis=0)} cells')

#     t = np.sum(img_driven, axis=1)
#     logging.info(f'img driven cells are driven by {t[t>0]} images')

#     ## save
#     vis_driven_re = {}
#     vis_driven_re['p_anova'] = p_anova
#     vis_driven_re['p_kruskal'] = p_kruskal
#     vis_driven_re['p_ttest'] = p_ttest
#     vis_driven_re['evoked'] = evoked
#     vis_driven_re['p_sig'] = p_sig
#     vis_driven_re['vis_driven'] = vis_driven
#     vis_driven_re['img_driven'] = img_driven

#     os.chdir(os.path.join(dir_inter, dir_sub))
#     print(os.getcwd())
#     with open('vis_driven.pickle', 'wb') as f:
#         pickle.dump(vis_driven_re, f)
    
#     # break

waiting for mat
V1_i1323_200720_003
isi_mode is grat_8ori_3isi (grating with 8 orientations and 3 ISI)
(103, 8, 3) (18, 1)
to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. 
 8 possible target orientations will cover all needed cells
grat_8ori_3isi


100%|██████████| 103/103 [00:00<00:00, 393.20it/s]

Z:\All_Staff\home\lan\Data\2P_images\mat_inter\V1_i1323_200720_003


In [16]:
np.array(base_cell_anova).flatten().shape

(142,)

In [11]:
stim_cell_anova.__len__(), stim_cell_anova[0].shape

(8, (18,))

# depre

In [37]:
# # meta = pd.read_excel(mat_inter_path + 'adp_dataset_master.xlsx')

# # date_str = str(200721)
# # mouse_str = meta.loc[meta['date'] == int(date_str), 'mouse'].values#[0]
# # area_str = meta.loc[meta['date'] == int(date_str), 'area'].values[0]
# # if len(mouse_str) > 1:
# #     print('duplicate dates with maybe different mouse. select which mouse?')
# # else:
# #     mouse_str = str(mouse_str[0])
# # print(mouse_str, date_str)

# try:
#     dir_sub = area_str + '_i' + mouse_str + '_' + date_str + '_cellpose'
#     dfof_trialwise = scipy.io.loadmat(os.path.join(mat_inter_path, dir_sub, 'resp_base_trialwise' + '.mat')) # dir_sub[:-7] delete caiman in dir_sub
# except:
#     dir_sub = area_str + '_i' + mouse_str + '_' + date_str + ''
#     dfof_trialwise = scipy.io.loadmat(os.path.join(mat_inter_path, dir_sub, 'resp_base_trialwise' + '.mat')) # dir_sub[:-7] delete caiman in dir_sub
# if len(dfof_trialwise['dfof_ad_trial'].shape) == 2: # if contains only one ISI, likely 250
#     mode = 'only one isi'
#     print(f'mode is {mode} (only one isi, grat_SF or bunny)')
#     print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0].shape)
#     dfof_ad_trial = dfof_trialwise['dfof_ad_trial'] # do not subtract baseline here: stim resp will be compared to base resp
#     dfof_base_trial = dfof_trialwise['dfof_base_trial']

# if len(dfof_trialwise['dfof_ad_trial'].shape) == 3: # if contains multiple ISI, likely inf-750-250
#     mode = 'grat_8ori_3isi'
#     print(f'mode is {mode} (grating with 8 orientations and 3 ISI)')
#     print(dfof_trialwise['dfof_ad_trial'].shape, dfof_trialwise['dfof_ad_trial'][0,0,0].shape)
#     dfof_tg_trial = dfof_trialwise['dfof_tg_trial'][:,:,0] # only use the first ISI, aka no adapter trials
#     dfof_base2_trial = dfof_trialwise['dfof_base2_trial'][:,:,0]
#     print('to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. \n 8 possible target orientations will cover all needed cells')

1323 200721
mode is grat_8ori_3isi (grating with 8 orientations and 3 ISI)
(39, 8, 3) (17, 1)
to find visually driven cells and image driven cells, we should take only no-adapter trials and use target response. 
 8 possible target orientations will cover all needed cells


In [38]:
# if mode == 'only one isi':
#     print(mode)

#     ncell = dfof_ad_trial.shape[0]
#     nstim = dfof_ad_trial.shape[1]

#     ## according to Ohki 2020
#     p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
#     p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
#     p_ttest = np.ones((ncell, nstim)) * np.nan
#     evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

#     for icell in tqdm(np.arange(ncell)):
#         base_cell_anova = np.concatenate([dfof_base_trial[icell, stim] for stim in range(nstim)])
#         stim_cell_anova = [] 
#         for istim in np.arange(nstim):
#             stim_cell_anova.append(np.array(dfof_ad_trial[icell, istim]).flatten())

#             base_cell = dfof_base_trial[icell, istim]
#             stim_cell = dfof_ad_trial[icell, istim]
#             _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
#             p_ttest[icell, istim] = p_ttest_i

#             evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
#             evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
#             evoked[icell, istim] = evoked_i
        
#         _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#         p_anova[icell] = p_anova_cell
#         _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#         p_kruskal[icell] = p_kruskal_cell

# elif mode == 'grat_8ori_3isi':
#     print(mode)

#     ncell = dfof_tg_trial.shape[0]
#     nstim = dfof_tg_trial.shape[1]

#     ## according to Ohki 2020
#     p_anova = np.ones((ncell, 1)) * np.nan # anova -> visually driven cells,  
#     p_kruskal = np.ones((ncell, 1)) * np.nan # t test -> certain image responsive cells,
#     p_ttest = np.ones((ncell, nstim)) * np.nan
#     evoked = np.ones((ncell, nstim)) * np.nan # amp thresh -> lower false positive rate

#     for icell in tqdm(np.arange(ncell)):
#         base_cell_anova = np.concatenate([dfof_base2_trial[icell, stim] for stim in range(nstim)])
#         stim_cell_anova = [] 
#         for istim in np.arange(nstim):
#             stim_cell_anova.append(np.array(dfof_tg_trial[icell, istim]).flatten())

#             base_cell = dfof_base2_trial[icell, istim]
#             stim_cell = dfof_tg_trial[icell, istim]
#             _, p_ttest_i = stats.ttest_ind(base_cell.flatten(), stim_cell.flatten(), equal_var=False, alternative='less')
#             p_ttest[icell, istim] = p_ttest_i

#             evoked_cell = (stim_cell - base_cell) / (base_cell + 1e-7)
#             evoked_i = np.mean(evoked_cell, axis=0) # trial averaged evoked resp
#             evoked[icell, istim] = evoked_i
        
#         _, p_anova_cell = stats.f_oneway(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#         p_anova[icell] = p_anova_cell
#         _, p_kruskal_cell = stats.kruskal(np.array(base_cell_anova).flatten(), *stim_cell_anova)
#         p_kruskal[icell] = p_kruskal_cell

grat_8ori_3isi


100%|██████████| 39/39 [00:00<00:00, 345.57it/s]


In [45]:
# # visually driven cells: pass (anova OR kruskal) AND amp threshold for >=1 image
# p_sig = 0.05
# vis_driven = ((p_anova < p_sig) | (p_kruskal < p_sig)) & (sum(evoked.T > 0.1) > 0).reshape(-1, 1)
# print(f'{vis_driven.sum()} cells are visually driven, \n\
#     proportion {np.round(vis_driven.sum()/ncell, 2)} out of {ncell} cells')

# # cells responsive to image i: pass visually driven (anova OR kruskal) AND t-test AND amp threshold for *this* image
# img_driven = vis_driven & (p_ttest < p_sig) & (evoked > 0.1)
# print(f'{img_driven.sum()} cells are image driven - with overlap between images, \n\
#     proportion {np.round(img_driven.sum() / (ncell*nstim), 2)} out of {ncell*nstim} cell-stim combos. \n\
#     1-N image evokes resp from {np.sum(img_driven, axis=0)} cells')

# t = np.sum(img_driven, axis=1)
# print(f'img driven cells are driven by {t[t>0]} images')

26 cells are visually driven, 
    proportion 0.67 out of 39 cells
29 cells are image driven - with overlap between images, 
    proportion 0.09 out of 312 cell-stim combos. 
    1-N image evokes resp from [4 2 6 1 3 2 5 6] cells
img driven cells are driven by [1 2 2 1 2 1 1 1 2 1 2 1 4 1 1 1 1 3 1] images


In [46]:
# vis_driven_re = {}
# vis_driven_re['p_anova'] = p_anova
# vis_driven_re['p_kruskal'] = p_kruskal
# vis_driven_re['p_ttest'] = p_ttest
# vis_driven_re['evoked'] = evoked
# vis_driven_re['p_sig'] = p_sig
# vis_driven_re['vis_driven'] = vis_driven
# vis_driven_re['img_driven'] = img_driven

# os.chdir(os.path.join(mat_inter_path, dir_sub))
# print(os.getcwd())

# with open('vis_driven.pickle', 'wb') as f:
#     pickle.dump(vis_driven_re, f)

\\duhs-user-nc1.dhe.duke.edu\dusom_glickfeldlab\All_Staff\home\lan\Data\2P_images\mat_inter\LM_i1323_200721
